# Linear Models
## Foundations of Machine Learning
## `! git clone https://www.github.com/DS3001/linearRegression`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def pn(x):
    print(x,'\n')

## Linear Models
- $k$NN and $k$MC illustrate the distinctions between regression and classification, and supervised and unsupervised learning
- In both cases, the number of parameters available for fitting the model is really limited --- just $k$ --- and they offer almost no explanation of their results
- Today we introduce *linear model*, which optimally weight the explanatory variables in order to predict the outcome variable
- These are extremely powerful and easily interpreted tools
- You can spend the rest of your life studying regression models (general linear models, quantile regression, kernel regression, etc.): This is an entry-level discussion that focuses on prediction and the intuition/mechanics of Ordinary Least Squares


## Vector Multiplication
- Suppose we have two vectors $x=(x_1,x_2,...x_K)$ and $b=(b_1,b_2,...,b_K)$ of equal length, $K$
- The *dot product* or *inner product* is
$$
x_1 b_1 + x_2 b_2 + ... + x_K b_K
$$

So we multiply the first two entries together, the second two together, and so on, then sum all the terms.
- Common notation for this is:
$$
x \cdot b = x^\top b = x'b = \langle x, b \rangle = \sum_{k=1}^K x_k b_k
$$
- Notice,
$$\text{cov}(x,y) = \dfrac{(x-\bar{x})\cdot (y-\bar{y})}{N} = \dfrac{\sum_{i=1}^N (x_i - \bar{x})(y_i - \bar{y})}{N}$$
so the covariance is a statistical version of the dot product.

## Dot Product Example
- Suppose `y = (3,-5,7)` and `x = (2,4,-6)`. How do we compute the dot product in Python?

In [ ]:
import numpy as np

y = (3,-5,7)
x = (2,4,-6)

print( np.inner(x,y), '\n') # Using Numpy

def dot(x,y): # Just example code in vanilla Python; avoid loops!
    n_x = len(x)
    n_y = len(y)
    if n_x == n_y:
        sum = 0
        for k in range(n_x):
            sum = sum + x[k]*y[k]
        return sum
    else:
        print('Lengths of arguments are not the same')
        raise

dot(x,y)

## Vector Multiplication [math]
- What "is" this thing? It is related to the angle between $x$ and $b$:
$$
\cos(\theta_{xb}) = \dfrac{x \cdot b}{||x|| \  ||b||}
$$
The dot product is the mathematical object that creates *angles* between objects in a space, and *covariance* in statistics ($cov(x,y) = (x-\bar{x})\cdot(y-\bar{y})/(N-1)$).
- Recall, $\cos( 90^\circ) = 0$ or $\cos(\pi/2) = 0$, so if the dot product between two vectors is zero, they are at a "right angle" to one another

## Matrix Multiplication
- If you stack rows of observations, you can multiply them all at once as follows:
$$
X \cdot b =  \left[\begin{array}{cccc} x_{11} & x_{12} & \dots & x_{1K} \\ x_{21} & x_{22} & \dots & x_{2K}  \\ \vdots & \vdots & \ddots & \vdots \\ x_{N1} & x_{N2} & \dots & x_{NK} \end{array} \right] \cdot \left( \begin{array}{c} b_1 \\ b_2 \\ \vdots \\ b_K \end{array}\right) = \left( \begin{array}{c} x_1 \cdot b \\ x_2 \cdot b \\ \vdots \\ x_N \cdot b\end{array} \right)
$$
- This is part of the motivation for "clean"/"tidy" data: We do calculations directly on the data frame. Having `NA`'s or ambiguity about what a row or column causes calculations to break down.


## Matrix Multiplication Example
- Suppose
$$ X = \left[ \begin{array}{ccc} 1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 9 \end{array} \right] $$
and
$$ b = \left( \begin{array}{c} 2 \\ 4 \\ 6 \end{array} \right)$$
- What is $X \cdot b$?

In [ ]:
X = np.matrix('1,2,3;4,5,6;7,8,9') # Making a matrix in numpy
print('Matrix X:')
pn(X)
b = np.array([2,4,6]) # Making a vector in numpy
print('Vector b:')
pn(b)
y = np.matmul(X,b) # Matrix multiplication in numpy
print('X dot b:')
pn(y)

## Linear Models: Setup
- The data include an $N \times K$ data matrix $X$ with $N$ observations and $K$ variables, and an $N$-length vector of outcomes, $y$
- We wish to use $X$ to explain $y$
- In particular, we want to explain $y$ by $X$ with a linear model,
$$
y = X \cdot b
$$
where $b$ is a $K$-length vector of *coefficients* or *weights*
- So for $b = (b_1, b_2, ..., b_K)$, variable $k$ is multiplied by $b_k$ to scale its contribution to the prediction of $y$

## Linear Models: Prediction
- To make a prediction for a new values $\hat{x} = (\hat{x}_1, \hat{x}_2, ..., \hat{x}_L)$, we compute the dot product:
$$
\hat{y} = \hat{x} \cdot b = \hat{x}_1 b_1  + \hat{x}_2 b_2  + ... + \hat{x}_K b_K 
$$
- The prediction is the straightforward part: Picking the weights $b$ is the hard part.

## Linear Models: Prediction
- In terms of matrix multiplication, a fitted model is a $\hat{b}$, and the predictions are created for data $\hat{X}$ as
$$
\hat{X} \cdot \hat{b} =  \left[\begin{array}{cccc} \hat{x}_{11} & \hat{x}_{12} & \dots & \hat{x}_{1K} \\ \hat{x}_{21} & \hat{x}_{22} & \dots & \hat{x}_{2K}  \\ \vdots & \vdots & \ddots & \vdots \\ \hat{x}_{N1} & \hat{x}_{N2} & \dots & \hat{x}_{NK} \end{array} \right] \left( \begin{array}{c} \hat{b}_1 \\ \hat{b}_2 \\ \vdots \\ \hat{b}_K \end{array}\right) = \left( \begin{array}{c} \hat{x}_1 \cdot \hat{b} \\ \hat{x}_2 \cdot \hat{b} \\ \vdots \\ \hat{x}_N \cdot \hat{b}\end{array} \right) = \left( \begin{array}{c} \hat{y}_1 \\ \hat{y}_2  \\ \vdots \\ \hat{y}_N \end{array} \right)
$$
- On a compute, these kinds of calulations are fast and efficient, and hardware like GPUs vastly speed up matrix/dot product calculations. Even for very complex models, a linear relationship between variables often appears somewhere (neural networks are non-linear compositions/nests of linear models).
- The next chunk of code gives a visual example of what we're talking about, for a simple model $\hat{y} = b_0 + b_1 x$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(500) # Set the seed for the random number generator
N = 30

x = 5 + 2*np.random.normal(0,1,size = N) # Create an x
eps = np.random.normal(0,3,size = N) # Create noise
b0 = -1 # Intercept coefficoent
b1 = 3 # Slope coefficient
y = b0 + b1*x + eps

def slr(x,y): # Single Linear Regression Function
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    b1 = np.inner(x-x_bar,y-y_bar)/np.inner(x-x_bar,x)
    b0 = y_bar - b1*x_bar
    y_hat = b0 + b1*x
    residuals = y - y_hat
    return({'b0':b0,'b1':b1,'y_hat':y_hat,'residuals':residuals})
    
reg = slr(x,y) # Run the regression

plt.scatter(x,y,label='Data')
plt.plot(x,reg['y_hat'],label='Regression Line')
plt.xlabel("X")
plt.ylabel("Y")
plt.legend(loc='lower right')
plt.title('Linear Regression')
plt.show()

## Case Study: Car Prices
- How does car age predict price?
- I'm going to interactively look at the data, clean outliers, take transformations, and regress, somewhat sloppily, then show you what the picture looks like if you don't do these steps
- This is a practical set of steps to take a
- My rough advice: Linear regressions is defined to approximate the **conditional expectation function** (CEF), $\mathbb{E}[y|x]$. It is "working" if your line is tracking with the average value of $y$ as $x$ varies, and is failing if the line is way off the mark (typically due to 1. outliers or 2. non-linearity of the CEF in $x$, requiring further transformation of the variables)

In [ ]:
import pandas as pd
import seaborn as sns
df = pd.read_csv('./data/USA_cars_datasets.csv') # Load the data
df0 = df # Let's keep the original data around for comparison purposes

In [ ]:
df.head() # Glance at the data

In [ ]:
df['price'].hist(bins=20) 

In [ ]:
df['age'] = max(df['year'])-df['year'] # Convert year to age
df['age'].hist(bins=20)

In [ ]:
sns.scatterplot(data=df,y='price',x='age') # We've got some outliers here

In [ ]:
# Take arcsinh transformation to rescale the variables
df['price_ihs'] = np.arcsinh(df['price'])
df['age_ihs'] = np.arcsinh(df['age'])
sns.scatterplot(data=df,y='price_ihs',x='age_ihs') # We've got some outliers here

In [ ]:
df['price_ihs'].plot.box() # Outliers below 9

In [ ]:
df['age_ihs'].plot.box() # Outliers above 4

In [ ]:
# Drop outliers:
df = df.loc[df['price_ihs']>9,:]
df = df.loc[df['age_ihs']<4,:]

In [ ]:
sns.scatterplot(data=df,y='price_ihs',x='age_ihs')

In [ ]:
x = df['age_ihs']
y = df['price_ihs']

coef = slr(x,y)

y_hat = coef['b0']+coef['b1']*x
plt.scatter(x,y,label='Data')
plt.plot(x,y_hat,label='Regression Line',color='black')
plt.xlabel("X")
plt.ylabel("Y")
plt.legend(loc='upper right')
plt.title('Linear Regression: Looks OK')

plt.show()

In [ ]:
# Why bother with all that data cleaning?
x = df0['age']
y = df0['price']

coef = slr(x,y)

y_hat = coef['b0']+coef['b1']*x
plt.scatter(x,y,label='Data')
plt.plot(x,y_hat,label='Regression Line',color='black')
plt.xlabel("X")
plt.ylabel("Y")
plt.legend(loc='lower left')
plt.title('Linear Regression: Looks Bad')

plt.show()

## Examples
- Predict car prices from attributes
- Predict Airbnb rental prices from housing features
- Predict probability of heart failure from patient characteristics
- Predict bond and sentence from defendant demographics and criminal record
- Predict student loan debt creation as a function of institution characteristics


## Errors/Residuals
- Fitting a linear model is based on minimizing the unexplained variation in the data
- Let $\hat{y}_i = x_i \cdot b$ be the prediction for observation $i$
- The *residual* or *error* for observation $i$ is
$$
e_i = y_i - \hat{y}_i = y_i - x_i \cdot b 
$$
This is how far off the in-sample prediction is, using the coefficients $b$ and variables $x_i$ for observation $i$ to make a prediction $\hat{y}_i$ --- how bad is our model at predicting values for data we already have?

## Sum-of-Squared-Error, `SSE`
- Some errors will generally be positive and some negative, but we want to count any error as undesirable, and larger errors as even worse failures. So, we square the error,
$$
e_i^2 = (y_i - \hat{y}_i)^2
$$
and sum over the observations,
$$
\text{SSE} = \sum_{i=1}^N e_i^2 = \sum_{i=1}^N (y_i - \hat{y}_i)^2
$$
to get the **Sum of Squared Error**. 
- This is often normalized as an average, to get the **mean squared error**,
$$
\text{MSE} = \dfrac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)^2,
$$
and often further normalized by taking the square root to get the **root mean square error**
$$
\text{RMSE} = \sqrt{ \dfrac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)^2}.
$$
- From a stats perspective there are subtle differences, but from a model-fitting perspective, these are all fundamentally the same thing: A metric of model performance based on squared error

In [ ]:
# The sum of the squares of the red lines is the SSE:
np.random.seed(500) # Set the seed for the random number generator
N = 30
x = 5 + 2*np.random.normal(0,1,size = N) # Create an x
eps = np.random.normal(0,3,size = N) # Create noise
b0 = -1 # Intercept coefficoent
b1 = 3 # Slope coefficient
y = b0 + b1*x + eps
reg = slr(x,y) # Run the regression
for i in range(len(x)):
    plt.vlines(x[i], y[i], reg['y_hat'][i], color='r') # Visualize residuals
plt.scatter(x,y,label='Data')
plt.plot(x,reg['y_hat'],label='Regression Line')
plt.xlabel("X")
plt.ylabel("Y")
plt.legend(loc='lower right')
plt.title('Visualizing SSE: Sum of Squared Red Lines')
plt.show()

## Other Model Metrics
- There is no reason you can't target other measures of model performance, like **mean absolute deviation** which is more robust to outliers,
$$ \text{MAD} = \dfrac{1}{N} \sum_{i=1}^N |y_i - \hat{y}_i|$$
or **worst absolute deviation** where worst-case prediction is the concern,
$$ \text{WAD} = \max_i |y_i - \hat{y}_i|$$
- There are (at least) hundreds of different metrics of model success besides `SSE`
- Why `SSE`? Probably because we can easily use calculus to minimize it.

## Minimizing the `SSE`
- The goal of *linear regression* is to pick $b$ to make $\text{SSE}(b)$ as small as possible. 
- Let's do this for a *single linear model* with a constant and one explanatory/feature variable $x_i$:
$$
y_i = b_0 \times 1 + b_1 \times x_i
$$
- Then the `SSE` is:
$$
\text{SSE}(b_0, b_1) = \sum_{i=1}^N (y_i - b_0 - b_1 x_i)^2
$$


## Optimization
- From my perpsective, the main reason to study calculus is to learn how to minimize/maximize and approximate functions
- To do interesting things in quantitative subjects, you typically need the mathematical background to maximize or minimize things like $\text{SSE}$ with respsect to $b_0$ and $b_1$
- Roughly, to minimize a function $f(b)$, you take the derivative with respect to $b$, $f'(b)$, set it equal to zero, and solve for $b^*$ --- this is called a *first order necessary condition*

## First-Order Conditions for Optimization
- If $\text{SSE}(b_0,b_1) = \sum_{i=1}^N(y_i-b_0 - b_1x_i)^2$, the necessary condition for $b_0$ is
$$
\sum_{i=1}^N-2(y_i-b_0-b_1 x_i) = 0
$$
and the necessary condition for $b_1$ is
$$
\sum_{i=1}^N -2(y_i - b_0 - b_1 x_i)x_i = 0
$$
- How do we simplify these?
- Define $\bar{x}$ as the mean of $x$,
$$
\bar{x} = \dfrac{1}{N} \sum_{i=1}^N x_i,
$$
and similarly for $\bar{y}$

## The First Condition
- We can simplify the first condition for $b_0$ as:
\begin{eqnarray*}
0 &=& \sum_{i=1}^N(y_i-b_0-b_1 x_i) \\
&=&  \sum_{i=1}^N y_i- \sum_{i=1}^N b_0-b_1 \sum_{i=1}^N x_i  \quad (\text{Distribute summation})\\
&=& \sum_{i=1}^N y_i- N b_0-b_1 \sum_{i=1}^N x_i \quad (\text{Sum $b_0$ $N$ times})\\
&=& \dfrac{\sum_{i=1}^N y_i}{N} -  b_0-b_1 \dfrac{\sum_{i=1}^N x_i}{N} \quad (\text{Divide by $N$})\\
0 &=& \bar{y} - b_0 - b_1 \bar{x} \quad (\text{Use definitions})
\end{eqnarray*}
implying $b_0^* = \bar{y} - b_1^* \bar{x}$.


## The Second Condition
- For the second condition for $b_1$:
\begin{eqnarray*}
0 &=& \sum_{i=1}^N (y_i - b_0 - b_1 x_i)x_i \\
&=& \sum_{i=1}^N (y_i - (\bar{y} - b_1 \bar{x}) - b_1 x_i)x_i \quad (\text{Substitute in $b_0^*$})\\
&=& \sum_{i=1}^N (y_i - \bar{y})x_i - b_1 (x_i -\bar{x})x_i \quad (\text{Distribute $x_i$, group terms})\\
0 &=& \sum_{i=1}^N (y_i - \bar{y})x_i - b_1 \sum_{i=1}^N (x_i -\bar{x})x_i \quad (\text{Distribute summation})\\
\end{eqnarray*}
implying
$$
b_1^* = \dfrac{\sum_{i=1}^N (y_i - \bar{y})x_i}{\sum_{i=1}^N (x_i -\bar{x})x_i}.
$$
(This is roughly the correlation between $x$ and $y$ divided by the variance of $x$)


## Single Linear Regression 
- Notice that the first condition can be written as
$$
0 = \sum_{i=1}^N (y_i - b_0 - b_1 x_i) 
  = \sum_{i=1}^N (y_i - \hat{y}_i)
 = \dfrac{1}{N} \sum_{i=1}^N e_i 
$$
so **the average error is equal to zero at the optimum**
- The second condition can be written as
$$
0 = \sum_{i=1}^N (y_i - b_0 - b_1 x_i)x_i 
= \sum_{i=1}^N (y_i - \hat{y}_i)x_i 
 = \sum_{i=1}^N e_i x_i 
 = e \cdot x 
$$
**The error term and explanatory variable are statistically uncorrelated, and at "right angles" to one another (orthogonal)**


## Single Linear Regression Function
- Here is an implementation of single linear regression, which returns a dictionary including the coefficients, the predicted values, and the residuals:

In [ ]:
def slr(x,y): # Single Linear Regression Function
    x_bar = np.mean(x) # Average of x's
    y_bar = np.mean(y) # Average of y's
    b1 = np.inner(x-x_bar,y-y_bar)/np.inner(x-x_bar,x) # Slope coefficient
    b0 = y_bar - b1*x_bar # Intercept coefficient
    y_hat = b0 + b1*x   # Compute predictions
    residuals = y - y_hat   # Compute residuals
    return({'b0':b0,'b1':b1,'y_hat':y_hat,'residuals':residuals})

## Partialing Out, Projection
- The regression breaks $y$ into two pieces:
\begin{eqnarray*}
y_i &=& (y_i - \hat{y}_i) + \hat{y}_i\\
&=& e_i + \hat{y}_i\\
\underbrace{y_i}_{\text{True value}} &=& \underbrace{e_i}_{\text{Error, residual}} + \underbrace{x_i \cdot b}_{\text{Model, prediction}}
\end{eqnarray*}
- But the residual from OLS averages to zero: It is uncorrelated with the prediction
- You can understand linear regression as removing the variation in $Y$ that can be explained by $X$ --- The residual contains all of the noise, the predictor $\hat{b} \cdot x$ contains all of the signal


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(500) # Set the seed for the random number generator
N = 30

x = 5 + 2*np.random.normal(0,1,size = N) # Create an x
eps = np.random.normal(0,3,size = N) # Create noise
b0 = -1 # Intercept coefficoent
b1 = 3 # Slope coefficient
y = b0 + b1*x + eps

def slr(x,y): # Single Linear Regression Function
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    b1 = np.inner(x-x_bar,y-y_bar)/np.inner(x-x_bar,x)
    b0 = y_bar - b1*x_bar
    y_hat = b0 + b1*x
    residuals = y - y_hat
    return({'b0':b0,'b1':b1,'y_hat':y_hat,'residuals':residuals})
    
reg = slr(x,y) # Run the regression
print('Coefficients: ',reg['b0'],reg['b1'])

# Plot the resuts:
for i in range(len(x)):
    plt.vlines(x[i], y[i], reg['y_hat'][i], color='r') # Visualize residuals


plt.scatter(x,y,label='Data')
plt.plot(x,reg['y_hat'],label='Regression Line')
plt.xlabel("X")
plt.ylabel("Y")
plt.legend(loc='lower right')
plt.title('Linear Regression')


In [ ]:
plt.subplot(1, 2, 1)
plt.scatter(x,reg['y_hat'])
plt.xlabel("X")
plt.ylabel("y_hat")
plt.title('Prediction: Signal')

plt.subplot(1, 2, 2)
plt.scatter(x,reg['residuals'])
plt.xlabel("X")
plt.ylabel("e")
plt.title('Residuals: Noise')

pd.DataFrame({'residuals':reg['residuals'],'x':x}).cov() # Compute correlation between e and x

## Another SLR Example
- Now that we understand SLR a bit better, let's do another case study, of Price versus Mileage

In [ ]:
df = df0 # Let's start over
df['mileage'].describe()
df['mileage_ihs'] = np.arcsinh(df['mileage'])
df['price_ihs'] = np.arcsinh(df['price'])
df['mileage_ihs'].plot.box()

In [ ]:
df['price_ihs'].plot.box()

In [ ]:
sns.scatterplot(data=df,y='price_ihs',x='mileage_ihs')

In [ ]:
sns.jointplot(data=df,y='price_ihs',x='mileage_ihs',kind='hex')

In [ ]:
# Drop outliers:
df = df.loc[df['mileage_ihs']>9,:] 
df = df.loc[df['mileage_ihs']<13,:]
df = df.loc[df['price_ihs']>9,:] 

In [ ]:
sns.scatterplot(data=df,x='price_ihs',y='mileage_ihs')

In [ ]:
x = df['mileage_ihs']
y = df['price_ihs']

coef = slr(x,y) # Single Linear Regression
print('Intercept: ',coef['b0'], '\n', ' Slope: ', coef['b1']) 

y_hat = coef['b0']+coef['b1']*x # Compute predictions

# Scatter plot of fit:
plt.scatter(x,y,label='Data')
plt.plot(x,y_hat,label='Regression Line',color='black')
plt.xlabel("Mileage, ihs")
plt.ylabel("Price, ihs")
plt.legend(loc='upper right')
plt.title('Linear Regression')

plt.show()

In [ ]:
# Without cleaning/feature engineering:
x = df0['mileage']
y = df0['price']

coef = slr(x,y)

y_hat = coef['b0']+coef['b1']*x

plt.scatter(x,y,label='Data')
plt.plot(x,y_hat,label='Regression Line',color='black')
plt.xlabel("Mileage")
plt.ylabel("Price")
plt.legend(loc='upper right')
plt.title('Linear Regression')
plt.show()

## Multiple Linear Regression
- The previous discussion is great: we have some geometic intuition about how linear regression works, its mathematical foundation, and its key properties
- Problem: It's only worked out for one regressor/feature/explanatory variable
- How do we extend this to multiple variables? i.e., the model
$$ y = \underbrace{b_0}_{\text{Intercept}} \times 1 + b_1 \times  x_1 + b_2 \times x_2 + ... + b_K \times x_K $$
- We adjust the `SSE` to include all the variables of interest:
$$ SSE = \sum_{i=1}^N (y_i - b_0 - b_1 x_{i1} - b_2 x_{i2} - ... - b_K x_{iK} )^2 = \sum_{i=1}^N (y_i - X_i \cdot b)^2$$
and maximize over $(b_0, b_1, ..., b_K)$.
- We won't go over the details analytically, but minimizing $(y-Xb)'(y-Xb)$ can be done computationally (gradient descent) or using linear algebra (the solution is $b^* = (X'X)^{-1}(X'y)$)

## Multiple Linear Regression Example
- Before we looked at `age` and `mileage` separately as predictors of price
- Now that we have MLR, we can combine then into a single model
$$ \text{price} = b_0 + b_{1} \times \text{age} + b_{2}\times\text{mileage}  $$

In [ ]:
# One-step multilinear regression:
def mlr(X,y): # Multiple linear regression, matrix algebra approach
    XpX = np.matmul(X.transpose(),X) # Compute X'X
    Xpy = np.matmul(X.transpose(),y) # Compute X'y
    b = np.linalg.solve(XpX, Xpy) # Solve normal equations
    y_hat = np.matmul(X,b) # Compute predictions
    residuals = y-y_hat # Compute residuals
    SSE =  np.inner(residuals,residuals) # Compute SSE
    rsq = 1 - SSE/np.inner( y-np.mean(y),y-np.mean(y)) # Compute Rsq
    return({'b':b,'y_hat':y_hat,'residuals':residuals,'rsq':rsq,'SSE':SSE})

df['(Intercept)'] = 1
X = df.loc[:,['(Intercept)','age_ihs','mileage_ihs'] ]
y = df['price_ihs']
reg = mlr(X,y)
print('MLR coefficients: ', reg['b']) # Same values!

## How does MLR work?
- This is the deep idea in regression
- Consider the $x_k$ whose coefficient you want to compute:
    1. Regress $y$ and $x_k$ on all of the other coefficients, yielding residuals $r_y$ and $r_{x_k}$
    2. Now regress $r_y$ on $r_{x_k}$ and a constant
- The slope coefficient for the procedure described above is the same as for MLR
- What does this mean? Linear regression "partials out" all of the variation in $y$ that can be explained by the other features, and the optimal weight $b_k$ reflects only the remaining variation in $y$ that can be explained by $x_k$ alone:
$$ 
b_k = \dfrac{ \text{cov}(r_y, r_{x_k})}{\sigma^2_{r_{x_k}}}
$$  
- We will return to this idea a lot to explain phenomena related to linear models

In [ ]:
# Let's do all the steps laboriously:

# Pick variables for analysis:
y = df['price_ihs']
x1 = df['mileage_ihs']
x2 = df['age_ihs']

reg1_y = slr(x1,y) # Regress y on x1
reg1_2 = slr(x1,x2) # Regress x2 on x1
y_temp = reg1_y['residuals'] # Extract the residual for y
x2_temp = reg1_2['residuals'] # Extract the residual for x2
reg_y_x2 = slr(x2_temp,y_temp) # Regress residuals on each other
print('Age coefficient: ', reg_y_x2['b1'])

reg2_y = slr(x2,y) # Regress y on x2
reg2_1 = slr(x2,x1) # Regress x1 on x2
y_resid = reg2_y['residuals'] # Extract the residual for y
x1_resid = reg2_1['residuals'] # Extract the residual for x1
reg_y_x1 = slr(x1_resid,y_resid) # Regress residuals on each other
print('Mileage coefficient: ', reg_y_x1['b1'])

# Compute intercept:
b0 = np.mean(y) - reg_y_x1['b1'] * np.mean(x1) - reg_y_x2['b1']*np.mean(x2)
print('Intercept: ', b0, '\n')

## Multiple Linear Regression 
- Once you have multiple linear regression, you can build much more complex models of phenomena
- We probably want to include $\text{age}^2$ and $\text{mileage}^2$ to control for non-linear aging effects:
$$ \text{price} = b_0 + b_{1} \times \text{age} + b_{2} \times\text{age}^2 + b_{3}\times\text{mileage} + b_{4}\times\text{mileage}^2 $$
- This gives a highly flexible and extensible way of modeling how features predict a target variable; you can only unlock the power of linear regression if you are willing to give it a large feature space to work with
- Eventually, we want to discipline this process by using data, not just making up models that run the risk of overfitting our data

In [ ]:
df.head()

In [ ]:
y = df['price_ihs']
df['mileage_ihs_sq'] = df['mileage_ihs']**2
df['age_ihs_sq'] = df['age_ihs']**2
df['(Intercept)'] = 1
vars = ['(Intercept)','mileage_ihs','mileage_ihs_sq','age_ihs','age_ihs_sq']
X = df.loc[:,vars]
X.head()

In [ ]:
reg = mlr(X,y) # Run multiple linear regression
pn(reg['b']) # Print coefficients 
reg['residuals'].plot.kde() # Plot residuals
pn(reg['rsq']) # R-squared measure of model fit

## The `sklearn.linear_model` Module
- Scikit-Learn has a linear regression object that can be used out-of-the box:
    - `from sklearn.linear_model import LinearRegression` to load the linear regression module
    - `myRegression = LinearRegression().fit(X, y)` fits 
- You'll see below that the results very similar to the `mlr` function above: They're using the same NumPy code "under the hood" to solve for the OLS coefficients, but with some additional feature engineering/normalization

In [ ]:
from sklearn.linear_model import LinearRegression # Import linear regression model

vars = ['mileage_ihs','mileage_ihs_sq','age_ihs','age_ihs_sq'] # This is a list of variables to use

X = df.loc[:,vars] # Construct data matrix
pn(X.head()) # Peek at data

reg = LinearRegression().fit(X, y) # Fit the linear model
pn(reg.intercept_) # Intercept value
pn(reg.coef_) # Regression coefficients
pn(reg.score(X, y)) # R squared measure

## Examples
- `./data/USA_cars_datasets.csv`
- `./data/airbnb_hw.csv`
- `./data/heart_failure/heart_failure_clinical_records_dataset.csv`
- `./data/pierce_county_house_sales.csv`

## Coefficient of Determination, $R^2$
- A natural question then is, how much noise is left to explain? How well does the model fit the data?
- There is a common statistic used to summarize how predictive $X$ is of $y$, using the OLS coefficients, called the *coefficient of determination* or $R^2$:
$$
R^2 = 1 - \dfrac{\sum_{i=1}^N (y_i - x_i \cdot \hat{b})^2}{\sum_{i=1}^N(y_i-\bar{y})^2}
$$
- That numerator is the sum-of-squared-error, evaluated at $\hat{b}$: How much error remains after trying to explain it using OLS?
- That denominator is the error of predicting $y_i$ with just the mean as the predictor, $\bar{y}$
- So the ratio is the reduction in the `SSE` by using the explanatory variables rather than just the sample mean
- This is a nice way to evaluate the model, but should not become an end in itself: Adding more variables always raises the $R^2$, but does not necessarily improve out-of-sample prediction
- $R^2$ isn't an end in itself

## Feature Engineering
- To fully leverage the power of regression, you have many options to expand the range of variables that the model can use to explain the variation in the data:
    - An **interaction term** is when you take two explanatory variables, say $x_1$ and $x_2$, and multiply them together to get a new explanatory variable, $z = x_1 x_2$, like $\text{mileage} \times \text{age}$
    - A **polynomial family** is when you take an explanatory variable, say $x$, and compute its powers $x^2$, $x^3$, ... , $x^K$. (There are many kinds of families besides polynomial.)
    - A **logarithmic transformation** or **inverse hyperbolic sine transformation**
    - A **maxmin normalization** or **z-score normalization**
    - More advanced tools like **principal components analysis** decomposition of the data or **splines** that create highly transformations of variables
    - Any combination of the above

## Model Selection
- Datasets often have dozens or thousands of variables, and we can create many more through transforming and interacting variables. Which variables go into a linear regression?
- In linear regression, our model is **underfit** if we use too few variables which cannot capture the complex relationships between the features and target variable, while our model is **overfit** if we are using too many variables which are exploiting too many unique features of the training data
- In statistics, it is an assumption that the analyst roughly knows the "true" data generating process, and properties of an estimator are derived under that assumption - statistics has few useful answers for how to pick the form of the model itself (e.g. Akaike Information Criterion, Bayesian Information Criterion, Mallows' $C_p$)
- In machine learning, we will learn some tools and techniques for model selection in a data-driven way later on in the course based on cross validation (e.g. LASSO)

## What, exactly, makes a model linear?
- A model is linear because the coefficients, $b$, enter the prediction as
$$
\hat{y} = b \cdot \hat{x} = b_1 \hat{x}_1 + b_2 \hat{x}_2 + ... b_J \hat{x}_J
$$
so that the relationship between $\hat{y}$ and each $\hat{x}_j$ is a linear one through $b_j$
- Taking non-linear transformations of the $x_j$'s simply gives new variables, like $\log(x_j)$ or $x_j^2$ -- *Transformations of the regressors don't make the model nonlinear*
- Likewise, if there was a coefficient written in a funky nonlinear way, like $\sqrt{ \text{asinh}(b_j)}$, you can simply relabel that coefficient as $b_j \leftarrow \sqrt{ \text{asinh}(b_j)}$ and make the model linear again.
- There *are* many interesting non-linear models, but they are often derived from a domain-specific theory and require additional work to understand

## The Classical Assumptions for OLS [stats]
- I often see people write the following: "The neccessary assumptions for linear regression are (i) linearity, (ii) homoskedasticity, (iii) conditional independence of errors, (iv) normality of errors"
- This generates confusion --- These are the assumptions for *OLS to be the best, linear, unbiased estimator (BLUE) of a hypothetical "true" $b_0$ in a finite sample, and the $z$-test to be correctly specified*
- We are not trying to estimate $b_0$, we are trying to do something else: find the *best linear predictor* of $y$ using the variables $X$. The above conditions are not necessary for that.
- You don't need permission to run OLS and make predictions, you need permission to run OLS and interpret the coefficients.

## General Linear Models
- You can spend the rest of your life studying linear models and their generalizations
- In many situations, there are additional restrictions on the environment that cannot always be satisfied by a linear model
    - The outcome/target variable might be a binary 0/1 outcome, so we are predicting the probability of something occuring. OLS might predict values less than zero or greater than one. Popular solutions are **Logit regression** or **Probit regression**
    - The outcome/target variable might be a non-negative integer, meaning that it is **count data**, like the number of earthquakes or cases of an illness. OLS won't predict counts. Popular solutions are **Poisson regression** or **negative binomial regression**.
- In these cases, we can estimate alternative models that impose restrictions on the outcome of the linear model, typically through maximum likelihood estimation or generalized linear modeling
- These aren't typically much more work to estimate, but we don't really have the time to cover all the nuances (and people typically estimate them only if the OLS model really breaks down in terms of predictive performance)